<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Sarcasm%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram K

### Connecting to Kaggle

In [ ]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [ ]:
!unzip /content/news-headlines-dataset-for-sarcasm-detection.zip

Archive:  /content/news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


### Importing Dependencies

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
dataset = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json", lines = True)

In [ ]:
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [ ]:
dataset["headline"][0]

'thirtysomething scientists unveil doomsday clock of hair loss'

### Taking X and Y

In [ ]:
X = dataset["headline"]

In [ ]:
y = dataset["is_sarcastic"]

### Splitting Data into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

### Tokenization

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 150

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<oov>")

In [ ]:
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
train_sequence = tokenizer.texts_to_sequences(X_train)

In [ ]:
train_padded = pad_sequences(train_sequence, maxlen = max_length, truncating = "post")

In [ ]:
test_sequence = tokenizer.texts_to_sequences(X_test)

In [ ]:
test_padded = pad_sequences(test_sequence, maxlen = max_length)

In [ ]:
type(y_train)

numpy.ndarray

### Define Basic Neural Networks

In [ ]:
model = tf.keras.models.Sequential([
                                    
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),

        tf.keras.layers.GlobalAveragePooling1D(),

        tf.keras.layers.Dense(512, activation = "relu"),

        tf.keras.layers.Dense(1, activation = "sigmoid")


])

In [ ]:
model.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               8704      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 169,217
Trainable params: 169,217
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_padded, y_train, validation_data = (test_padded, y_test), epochs = 10)

Epoch 1/10
716/716 [==============================] - 8s 6ms/step - loss: 0.5700 - accuracy: 0.6797 - val_loss: 0.3889 - val_accuracy: 0.8241
Epoch 2/10
716/716 [==============================] - 4s 6ms/step - loss: 0.3235 - accuracy: 0.8649 - val_loss: 0.3495 - val_accuracy: 0.8417
Epoch 3/10
716/716 [==============================] - 4s 6ms/step - loss: 0.2580 - accuracy: 0.8960 - val_loss: 0.3347 - val_accuracy: 0.8519
Epoch 4/10
716/716 [==============================] - 4s 6ms/step - loss: 0.2165 - accuracy: 0.9138 - val_loss: 0.3411 - val_accuracy: 0.8538
Epoch 5/10
716/716 [==============================] - 4s 6ms/step - loss: 0.1849 - accuracy: 0.9282 - val_loss: 0.4154 - val_accuracy: 0.8346
Epoch 6/10
716/716 [==============================] - 5s 6ms/step - loss: 0.1674 - accuracy: 0.9357 - val_loss: 0.3924 - val_accuracy: 0.8482
Epoch 7/10
716/716 [==============================] - 5s 7ms/step - loss: 0.1459 - accuracy: 0.9452 - val_loss: 0.4195 - val_accuracy: 0.8487
Epoch 

### Classifying the new text

In [ ]:
sentences = ["Oh is it?", "Silence is golden. Duct tape is silver."]

In [ ]:
predict_sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
padded_seq = pad_sequences(sequences = predict_sequences, maxlen = max_length, truncating = "post")

In [ ]:
model.predict(padded_seq)

array([[0.0086363],
       [0.664812 ]], dtype=float32)

### Implementing LSTM

In [ ]:
model1 = tf.keras.models.Sequential([
                                     
         tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),

         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),

         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),

         tf.keras.layers.Dense(64, activation = "relu"),

         tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 128)          41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 308,609
Trainable params: 308,609
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [ ]:
model1.fit(train_padded, y_train, validation_data = (test_padded, y_test), epochs = 10)

Epoch 1/10
716/716 [==============================] - 119s 153ms/step - loss: 0.5487 - accuracy: 0.6962 - val_loss: 0.3768 - val_accuracy: 0.8367
Epoch 2/10
716/716 [==============================] - 108s 151ms/step - loss: 0.3001 - accuracy: 0.8763 - val_loss: 0.3525 - val_accuracy: 0.8499
Epoch 3/10
716/716 [==============================] - 108s 150ms/step - loss: 0.2125 - accuracy: 0.9178 - val_loss: 0.3720 - val_accuracy: 0.8456
Epoch 4/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1662 - accuracy: 0.9387 - val_loss: 0.4918 - val_accuracy: 0.8377
Epoch 5/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1313 - accuracy: 0.9548 - val_loss: 0.4719 - val_accuracy: 0.8389
Epoch 6/10
716/716 [==============================] - 108s 151ms/step - loss: 0.1057 - accuracy: 0.9641 - val_loss: 0.5486 - val_accuracy: 0.8438
Epoch 7/10
716/716 [==============================] - 108s 151ms/step - loss: 0.0783 - accuracy: 0.9757 - val_loss: 0.6350 -

In [ ]:
model1.predict(padded_seq)

array([[0.00936779],
       [0.2414308 ]], dtype=float32)